# Analysis Tests

## Setup and Helper Functions

In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
from fau_colors import cmaps, register_fausans_font

from pepbench.datasets import EmpkinsDataset
from pepbench.io import load_challenge_results_from_folder

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

In [ ]:
deploy_type = "local"
config_dict = json.load(Path("./config.json").open(encoding="utf-8"))

guardian_base_path = Path(config_dict[deploy_type]["guardian_path"])
empkins_base_path = Path(config_dict[deploy_type]["empkins_path"])

In [ ]:
results_path = Path("../experiments/pep_algorithm_benchmarking/results")

In [ ]:
dataset_empkins = EmpkinsDataset(empkins_base_path, use_cache=True, only_labeled=True)
dataset_empkins

In [ ]:
results_type = "empkins_dataset_both_algorithms"

folder_path = results_path.joinpath(results_type)
folder_path

In [ ]:
results_agg, results_single, results_per_sample = load_challenge_results_from_folder(
    folder_path,
    index_cols_single=["participant", "condition", "phase"],
    index_cols_per_sample=["participant", "condition", "phase"],
)

In [ ]:
mean_agg = (
    results_per_sample.filter(like="metric", axis=1)
    .groupby(["q_wave_algorithm", "b_point_algorithm", "outlier_correction_algorithm"])
    .agg(["mean", "std"])
    .sort_values(by=("absolute_error_per_sample_ms", "metric", "mean"))
)

best_algo = mean_agg.iloc[0].name
best_algo

In [ ]:
results_per_sample.loc[best_algo]

In [ ]:
fig, ax = plt.subplots()

sns.boxplot(
    data=results_per_sample.loc[best_algo].reset_index(), x="phase", y=("absolute_error_per_sample_ms", "metric"), ax=ax
)
# sns.stripplot(data=results_per_sample.loc[best_algo].reset_index(), x="phase", y=("error_per_sample_ms", "metric"), ax=ax)

fig.tight_layout()